<a href="https://colab.research.google.com/github/maytlim/doh_data/blob/main/vacancies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -s -O https://raw.githubusercontent.com/maytlim/doh_data/main/landline.py

In [2]:
import requests
import pandas as pd
import datetime
import landline
import gdown

In [3]:
  pd.options.display.float_format = '{:,.0f}'.format

In [4]:
try:
  import pdfx
except ModuleNotFoundError:
  !pip -q install pdfx
  import pdfx

In [5]:
#Code from https://gist.github.com/korakot/51a917e1f53891d53be223439b0f75c1
from google.colab import auth
auth.authenticate_user()  # must authenticate

'''list all ids of files directly under folder folder_id'''
def folder_list(folder_id):
  from googleapiclient.discovery import build
  gdrive = build('drive', 'v3', cache_discovery=False).files()
  #gdrive = build('drive', 'v3').files()
  res = gdrive.list(q="'%s' in parents" % folder_id).execute()
  return [f['id'] for f in res['files']]

'''download all files from a gdrive folder to current directory'''
def folder_download(folder_id):
  for fid in folder_list(folder_id):
    !gdown -q --id $fid

In [6]:
ddrop_readme = requests.get('http://bit.ly/DataDropPH').url
ddrop_readme = ddrop_readme.split('/')[-1].split('?')[0]
#print(ddrop_readme)
folder_download(ddrop_readme)

In [7]:
readme_pdf = !ls *.pdf
readme_pdf = readme_pdf[0][1:-1]
readme_pdf
pdfx_tmp = pdfx.PDFx(readme_pdf)
ddlinks = pdfx_tmp.get_references_as_dict()

In [8]:
month_day = readme_pdf.split('_')
month = int(month_day[0][-2:])
day = int(month_day[1][:2])
month, day
year = 2021
print('DOH DataDrop Release:', year, month, day)

DOH DataDrop Release: 2021 4 2


In [9]:
downloaded = False
for ddlink in ddlinks['url']:
  if downloaded == False:
    if ddlink[:15] == 'https://bit.ly/' and ddlink[-3:] != 'ive':
      datadrop = requests.get(ddlink).url
      datadrop = datadrop.split('/')[-1].split('?')[0]
      folder_download(datadrop)
      print('Downloading DOH Data Drop...')
      downloaded = True
  else:
    continue

In [10]:
data_date = datetime.datetime(year, month, day)
#data_dir ='DOH COVID Data Drop_'  + data_date.strftime('%Y%m%d') + '/'

In [11]:
landline_masterlist = landline.make_masterlist()

In [12]:
# hospital daily patient census
cap_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 05 DOH Data Collect - Daily Report.csv'
# hospital weekly inventory of supplies
sup_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 06 DOH Data Collect - Weekly Report.csv'
print('weekly supplies inventory:', sup_filename)
print('daily capability count:', cap_filename)
hosp_code = 'hfhudcode' 
#hosp_name = 'cfname'

weekly supplies inventory: DOH COVID Data Drop_ 20210402 - 06 DOH Data Collect - Weekly Report.csv
daily capability count: DOH COVID Data Drop_ 20210402 - 05 DOH Data Collect - Daily Report.csv


In [13]:
#load capability data
df_cap = pd.read_csv(cap_filename)
df_cap['cap_reportdate'] = pd.to_datetime(df_cap['reportdate'])
df_cap.drop(['updateddate', 'addeddate', 'reportdate', 'id', 'north_coord', 'east_coord'], axis=1, inplace=True)
df_cap[hosp_code] = df_cap[hosp_code].apply(lambda x: x.strip())

In [14]:
!gdown -q 'https://raw.githubusercontent.com/maytlim/doh_data/main/nhfr.json'

In [15]:
df_nhfr = pd.read_json('nhfr.json')
print('Refer to https://nhfr.doh.gov.ph/ for the latest data')

Refer to https://nhfr.doh.gov.ph/ for the latest data


In [16]:
df_cap = df_cap.merge(how='left', right=df_nhfr, left_on='hfhudcode', right_on='Health Facility Code')

In [17]:
#load supplies data
df_sup = pd.read_csv(sup_filename)
df_sup['sup_reportdate'] = pd.to_datetime(df_sup['reportdate'])
df_sup.drop(['updateddate', 'addeddate', 'reportdate', 'id'], axis=1, inplace=True)
df_sup[hosp_code] = df_sup[hosp_code].apply(lambda x: x.strip())

In [18]:
# quarantine facility daily patient census
quarantine_daily = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 08 Quarantine Facility Data - Daily Report.csv')
# quarantine facility weekly inventory of supplies
quarantine_weekly = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 09 Quarantine Facility Data - Weekly Report.csv')

In [19]:
# HCW availability
#baseline_v3 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 10 DOH Data Collect v3 - Baseline.csv')
# mechvent, swabbing, ipc
#baseline_v4 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 11 DOH Data Collect v4 - Baseline.csv')
# quarantine and isolation facilities
#ttmf = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 12 DDC TTMF Tracker v1 - Baseline.csv')

In [20]:
#print('{0}\n{1}'.format(df_cap.keys(), df_sup.keys()))
#print('{0}\n{1}'.format(quarantine_weekly.keys(), quarantine_daily.keys()))

In [21]:
reg_lookup = {'NCR': 'NATIONAL CAPITAL REGION (NCR)', 'ARMM': 'AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM)', 
              'CAR': 'CORDILLERA ADMINISTRA TIVE REGION (CAR)', 'I': 'REGION I (ILOCOS REGION)', 
              'II': 'REGION II (CAGAYAN VALLEY)', 'III': 'REGION III (CENTRAL LUZON)', 
              'IV-A': 'REGION IV-A (CALABAR ZON)', 'IV-B': 'REGION IV-B (MIMAROPA)', 
              'V': 'REGION V (BICOL REGION)', 'VI': 'REGION VI (WESTERN VISAYAS)', 
              'VII': 'REGION VII (CENTRAL VISAYAS)', 'VIII': 'REGION VIII (EASTERN VISAYAS)', 
              'IX': 'REGION IX (ZAMBOANGA PENINSULA)', 'X': 'REGION X (NORTHERN MINDANAO)', 
              'XI': 'REGION XI (DAVAO REGION)', 'XII': 'REGION XII (SOCCSKSA RGEN)', 
              'XIII': 'REGION XIII (CARAGA)'}

In [22]:
prefix = ['ics', 'resdoc', 'nurse', 'medtech', 'respthe', 'radtech', 'support']
suffix = ['_total', '_er', '_icu', '_ward'] 
triage = ['resdoc_triage', 'nurse_triage', 'support_triage']
added = ['Augmen_total']

In [23]:
def day_ago_from(n, from_date):
  showdate = from_date - datetime.timedelta(hours=24)*n
  return showdate.strftime('%Y-%m-%d')

In [24]:
def show_filtered(df_cap, days_ago, region='NATIONAL CAPITAL REGION (NCR)'):
  filter = (df_cap.mechvent_v > 0) & \
    (df_cap.cap_reportdate == day_ago_from(days_ago, data_date)) & \
    (df_cap.region == region) & \
    (df_cap.icu_v > 0) & \
    (df_cap.icu_o > 0)
  df_vacant = df_cap[filter][['cap_reportdate', 'cfname', 'city_mun', 'icu_v', 'isolbed_v', 'mechvent_v', 'beds_ward_v', 
                            'icu_o', 'isolbed_o', 'mechvent_o', 'beds_ward_o', 
                            'Landline Number', 'Landline Number 2', 'Fax Number', 'Ownership Major Classification']]
  df_vacant.rename({'Landline Number': 'landline', 'Landline Number 2': 'landline2', 'Fax Number': 'fax', 'Ownership Major Classification':'ownership'}, axis=1, inplace=True)
  df_vacant['landline'] = df_vacant['landline'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['landline2'] = df_vacant['landline2'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['fax'] = df_vacant['fax'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant.sort_values(by='icu_v', ascending=False, inplace=True)
  df_vacant.reset_index(inplace=True)
  return df_vacant

In [25]:
show_filtered(df_cap, days_ago=3)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,18909,2021-03-30,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,69,86,36,0,31,314,24,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
1,253581,2021-03-30,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,10,26,10,0,10,10,5,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
2,265308,2021-03-30,JUSTICE JOSE ABAD SANTOS GENERAL HOSPITAL,BINONDO,7,8,5,0,5,23,1,2,(02) 8254 2453,(02) 5310 1984,None,Government
3,143381,2021-03-30,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,6,2,1,1,4,9,10,15,(02) 8716 1853,None,(02) 8716 1853,Private
4,193384,2021-03-30,PASAY CITY GENERAL HOSPITAL,PASAY CITY,6,7,1,0,6,11,3,28,(02) 8833 6022,(02) 8831 3285,(02) 8551 3735,Government
5,127266,2021-03-30,DR. VICTOR R. POTENCIANO MEDICAL CENTER,CITY OF MANDALUYONG,5,4,4,0,3,5,4,30,(02) 8464 9999,None,(02) 8532 1764,Private
6,223916,2021-03-30,ST. LUKE?S MEDICAL CENTER - GLOBAL CITY,TAGUIG CITY,5,2,4,7,17,5,15,99,(02) 8789 7700,None,None,Private
7,155447,2021-03-30,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,4,9,4,0,25,72,0,0,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
8,37088,2021-03-30,NATIONAL CHILDREN?S HOSPITAL,QUEZON CITY,3,49,5,0,10,28,10,0,(02) 8724 0656,None,(02) 8721 9139,Government
9,42495,2021-03-30,"CAPITOL MEDICAL CENTER, INC.",QUEZON CITY,3,5,3,1,6,10,6,9,(02) 8372 3825,(02) 8938 7789,(02) 3411 4320,Private


In [26]:
show_filtered(df_cap, days_ago=2)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,18910,2021-03-31,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,65,85,35,0,35,315,25,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
1,253582,2021-03-31,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,8,26,10,0,12,12,5,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
2,127267,2021-03-31,DR. VICTOR R. POTENCIANO MEDICAL CENTER,CITY OF MANDALUYONG,7,15,4,0,2,9,5,31,(02) 8464 9999,None,(02) 8532 1764,Private
3,265309,2021-03-31,JUSTICE JOSE ABAD SANTOS GENERAL HOSPITAL,BINONDO,7,3,5,0,5,28,1,2,(02) 8254 2453,(02) 5310 1984,None,Government
4,122369,2021-03-31,TONDO MEDICAL CENTER,TONDO I / II,6,22,1,0,12,22,5,82,(02) 8522 1174,(02) 8522 9244,(02) 8252 8661,Government
5,155448,2021-03-31,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,5,10,1,0,21,71,3,0,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
6,143382,2021-03-31,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,5,2,1,1,5,9,10,15,(02) 8716 1853,None,(02) 8716 1853,Private
7,97562,2021-03-31,PHILIPPINE HEART CENTER,QUEZON CITY,3,63,2,0,10,65,5,0,(02) 8925 2401,(02) 8922 0551,(02) 8922 0551,Government
8,223917,2021-03-31,ST. LUKE?S MEDICAL CENTER - GLOBAL CITY,TAGUIG CITY,3,2,7,3,19,5,12,103,(02) 8789 7700,None,None,Private
9,203493,2021-03-31,JOSE R. REYES MEMORIAL MEDICAL CENTER,SANTA CRUZ,3,19,4,19,4,46,3,44,(02) 8711 9491,(02) 8711 9498,(02) 8732 1077,Government


In [27]:
show_filtered(df_cap, days_ago=1)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,18911,2021-04-01,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,67,73,34,0,33,327,26,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
1,155449,2021-04-01,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,8,19,4,0,21,62,0,0,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
2,265310,2021-04-01,JUSTICE JOSE ABAD SANTOS GENERAL HOSPITAL,BINONDO,7,29,5,0,5,25,1,2,(02) 8254 2453,(02) 5310 1984,None,Government
3,122370,2021-04-01,TONDO MEDICAL CENTER,TONDO I / II,5,19,1,0,13,18,6,89,(02) 8522 1174,(02) 8522 9244,(02) 8252 8661,Government
4,143383,2021-04-01,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,5,3,2,1,5,8,10,15,(02) 8716 1853,None,(02) 8716 1853,Private
5,127268,2021-04-01,DR. VICTOR R. POTENCIANO MEDICAL CENTER,CITY OF MANDALUYONG,5,7,4,0,3,8,5,32,(02) 8464 9999,None,(02) 8532 1764,Private
6,253583,2021-04-01,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,5,21,10,0,15,41,5,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
7,223918,2021-04-01,ST. LUKE?S MEDICAL CENTER - GLOBAL CITY,TAGUIG CITY,4,2,2,8,18,5,17,98,(02) 8789 7700,None,None,Private
8,119910,2021-04-01,DR. FE DEL MUNDO MEDICAL CENTER,QUEZON CITY,3,0,2,0,3,0,3,35,(02) 8712 0845,None,(02) 8712 5796,Private
9,37090,2021-04-01,NATIONAL CHILDREN?S HOSPITAL,QUEZON CITY,3,30,3,0,10,47,12,0,(02) 8724 0656,None,(02) 8721 9139,Government


In [28]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['II'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,181333,2021-04-01,REGION II TRAUMA AND MEDICAL CENTER (VETE...,BAYOMBONG (CAPITAL),3,23,2,7,6,32,6,21,+63 783212090,+63 788053560,None,Government


In [29]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['IV-A'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,228808,2021-04-01,DE LA SALLE UNIVERSITY MEDICAL CENTER,CITY OF DASMARIÑAS,5,6,3,0,3,32,13,0,(02) 8988 3100,+63 464818000,None,Private
1,171489,2021-04-01,BATANGAS MEDICAL CENTER,BATANGAS CITY (CAPITAL),4,22,29,0,7,10,1,35,+63 437408307,None,+63 437230165,Government
2,298360,2021-04-01,GENERAL EMILIO AGUINALDO MEMORIAL HOSPITAL,TRECE MARTIRES CITY (CAPITAL),4,17,10,12,2,8,0,7,+63 464190063,None,None,Government
3,305995,2021-04-01,"GLOBAL CARE MEDICAL CENTER OF CANLUBANG, INC.",CITY OF CALAMBA,4,1,3,5,1,0,0,19,+63 495205626,None,None,Private
4,325865,2021-04-01,"MARY MEDIATRIX MEDICAL CENTER, INC.",LIPA CITY,4,1,7,0,7,0,0,22,+63 437736800,+63 437736861,+63 437736861,Private
5,177506,2021-04-01,"ST. JAMES HOSPITAL, INC.",CITY OF SANTA ROSA,3,0,3,0,1,4,0,0,+63 495341254,None,None,Private
6,89943,2021-04-01,LIPA MEDIX MEDICAL CENTER,LIPA CITY,2,4,6,4,2,4,0,4,+63 437563008,None,None,Private
7,139007,2021-04-01,MEDICAL CENTER WESTERN BATANGAS,BALAYAN,2,3,1,3,1,3,0,3,+63 434071103,None,+63 434071101,Private
8,188958,2021-04-01,"LOS BA?OS DOCTORS HOSPITAL AND MEDICAL CENTER,...",LOS BAÑOS,2,10,1,0,3,20,1,0,+63 495360100,None,+63 495361225,Private
9,226631,2021-04-01,UNIHEALTH-SOUTHWOODS HOSPITAL AND MEDICAL CENT...,CITY OF BIÑAN,2,18,15,6,3,3,0,9,+63 498616385,None,None,Private


In [30]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['III'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,118279,2021-04-01,JOSE B. LINGAD MEMORIAL REGIONAL HOSPITAL,CITY OF SAN FERNANDO (CAPITAL),18,0,18,19,64,0,10,226,+63 459613544,None,None,Government
1,296460,2021-04-01,BATAAN GENERAL HOSPITAL AND MEDICAL CENTER,CITY OF BALANGA (CAPITAL),9,25,14,4,30,74,34,8,+63 472372269,None,None,Government
2,43313,2021-04-01,DR. PAULINO J. GARCIA MEMORIAL RESEARCH & MEDI...,CABANATUAN CITY,7,37,12,24,9,11,4,32,+63 444638286,+63 444639937,None,Government
3,11544,2021-04-01,BULACAN MEDICAL CENTER,CITY OF MALOLOS (CAPITAL),5,85,48,34,25,19,2,66,+63 447910630,None,None,Government
4,180786,2021-04-01,JAMES L. GORDON MEMORIAL HOSPITAL,OLONGAPO CITY,3,42,4,2,1,27,1,6,+63 476021229,+63 476021230,+63 476021228,Government
5,316361,2021-04-01,"GRACE GENERAL HOSPITAL, INC.",CITY OF SAN JOSE DEL MONTE,3,2,1,0,1,12,0,0,+63 445194838,None,None,Private
6,1362,2021-04-01,PREMIERE MEDICAL CENTER,CABANATUAN CITY,2,18,2,15,2,2,0,6,+63 444637845,None,None,Private
7,180239,2021-04-01,"CENTRAL LUZON DOCTORS? HOSPITAL, INC.",CITY OF TARLAC (CAPITAL),2,33,5,0,6,18,0,0,+63 459820806,None,None,Private
8,202947,2021-04-01,PRES. RAMON MAGSAYSAY MEMORIAL HOSPITAL,IBA (CAPITAL),2,6,3,25,6,31,1,155,+63 478117212,None,None,Government
9,74724,2021-04-01,"OUR LADY OF MT. CARMEL MEDICAL CENTER, INC.",CITY OF SAN FERNANDO (CAPITAL),1,9,1,0,6,17,6,0,+63 458605977,None,+63 454551416,Private


In [31]:
roi = reg_lookup['NCR']
#roi = reg_lookup['II']
#roi = reg_lookup['IV-A']
filter = (df_cap.cap_reportdate == day_ago_from(1, data_date)) & (df_cap.region == roi)
occupied = df_cap[filter]['icu_o'].sum()
vacant = df_cap[filter]['icu_v'].sum()
pct_used = (occupied / (vacant + occupied)) * 100